In [9]:
import os
import face_recognition
import numpy as np
import pandas as pd

# Directory containing known faces organized by person name in subdirectories
KNOWN_FACES_DIR = "known_faces"

# CSV file to save training data
CSV_FILE = "face_data.csv"

# Initialize lists for storing data
face_encodings = []
face_names = []
face_authorized_status = []

# Load existing data from the CSV file, if it exists
if os.path.exists(CSV_FILE):
    face_data = pd.read_csv(CSV_FILE)
else:
    face_data = pd.DataFrame(columns=["Name", "Authorized"])

# Loop through each person in the known_faces directory
for person_name in os.listdir(KNOWN_FACES_DIR):
    person_dir = os.path.join(KNOWN_FACES_DIR, person_name)

    # Check if it's a directory
    if os.path.isdir(person_dir):
        # Check if the person already exists in the CSV file
        if person_name in face_data["Name"].values:
            print(f"{person_name} is already in the CSV file. Retraining with new images...")
            face_data = face_data[face_data["Name"] != person_name]  # Remove existing record to update
        else:
            # Prompt user for authorized/unauthorized status
            while True:
                status_input = input(f"Is {person_name} authorized? (yes/no): ").strip().lower()
                if status_input in ('yes', 'no'):
                    is_authorized = True if status_input == 'yes' else False
                    break
                else:
                    print("Invalid input. Please enter 'yes' or 'no'.")

        # Process each image file in the person's directory
        for filename in os.listdir(person_dir):
            image_path = os.path.join(person_dir, filename)

            # Check if the file is an image
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                try:
                    # Load the image file
                    image = face_recognition.load_image_file(image_path)
                    
                    # Get face encodings
                    encodings = face_recognition.face_encodings(image)
                    
                    # If an encoding is found, add it to our lists
                    if encodings:
                        print(f"Processed image: {image_path}")
                        face_encodings.append(encodings[0])  # Save only one encoding per person
                except Exception as e:
                    print(f"Error processing {image_path}: {e}")
        
        # Save the person's data (single record)
        face_names.append(person_name)
        face_authorized_status.append(is_authorized)

# Convert the new data to a DataFrame
new_face_data = pd.DataFrame({
    "Name": face_names,
    "Authorized": face_authorized_status
})

# Combine existing and new data, ensuring there are no duplicates
updated_face_data = pd.concat([face_data, new_face_data], ignore_index=True)

# Save the combined data back to the CSV file
updated_face_data.to_csv(CSV_FILE, index=False)

# Save face encodings to a NumPy file (optional, if needed later for detection)
face_encodings = np.array(face_encodings)
np.save("face_encodings.npy", face_encodings)

print(f"Training complete. Data saved to {CSV_FILE}.")

Is Akhash authorized? (yes/no):  no


Processed image: known_faces/Akhash/image_7.jpeg
Processed image: known_faces/Akhash/image_0.jpeg
Processed image: known_faces/Akhash/image_1.jpeg
Processed image: known_faces/Akhash/image_2.jpeg
Processed image: known_faces/Akhash/image_3.jpeg


Is Arun authorized? (yes/no):  yes


Processed image: known_faces/Arun/image34.jpeg
Processed image: known_faces/Arun/image22.jpeg
Processed image: known_faces/Arun/image18.jpeg
Processed image: known_faces/Arun/image38.jpeg
Processed image: known_faces/Arun/image14.jpeg
Processed image: known_faces/Arun/image15.jpeg
Processed image: known_faces/Arun/image19.jpeg
Processed image: known_faces/Arun/image1.jpeg
Processed image: known_faces/Arun/image23.jpeg
Processed image: known_faces/Arun/image35.jpeg
Processed image: known_faces/Arun/image28.jpeg
Processed image: known_faces/Arun/image12.jpeg
Processed image: known_faces/Arun/image32.jpeg
Processed image: known_faces/Arun/image24.jpeg
Processed image: known_faces/Arun/image6.jpeg
Processed image: known_faces/Arun/image7.jpeg
Processed image: known_faces/Arun/image25.jpeg
Processed image: known_faces/Arun/image33.jpeg
Processed image: known_faces/Arun/image13.jpeg
Processed image: known_faces/Arun/image29.jpeg
Processed image: known_faces/Arun/image10.jpeg
Processed image:

In [ ]:
import cv2
import face_recognition
import numpy as np
import pandas as pd
import psycopg2
from datetime import datetime

# PostgreSQL connection details (replace with your details)
db_config = {
    'host': '34.47.193.205',
    'port': '5432',
    'database': 'hons',
    'user': 'postgres',
    'password': 'hons_postgres'
}

# CSV file containing known faces and authorization status
CSV_FILE = "face_data.csv"

# Load face data from CSV
face_data = pd.read_csv(CSV_FILE)
known_names = face_data["Name"].values
known_authorized_status = face_data["Authorized"].values

# Load face encodings from the saved numpy file
known_face_encodings = np.load("face_encodings.npy", allow_pickle=True)

# Function to save detected person data to the PostgreSQL database
def save_detection_to_db(name, authorized, timestamp):
    try:
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()

        # Insert detection record into the person_data table
        insert_query = """
        INSERT INTO person_data (name, authorized, timestamp)
        VALUES (%s, %s, %s)
        """
        cursor.execute(insert_query, (name, authorized, timestamp))
        
        conn.commit()
        cursor.close()
        conn.close()

        print(f"Data saved: {name}, {'Authorized' if authorized else 'Unauthorized'}, {timestamp}")
    except Exception as e:
        # Do not print exception, silently handle it
        pass

# Initialize the webcam
video_capture = cv2.VideoCapture(0)

print("Face detection started. Press 'q' to quit.")

while True:
    # Capture frame-by-frame from the webcam
    ret, frame = video_capture.read()
    if not ret:
        print("Failed to capture image.")
        break
    
    # Resize the frame for faster processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
    
    # Detect faces and compute face encodings in the current frame
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
    
    # Loop through each face found in the frame
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        try:
            # Compare the detected face with known faces
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.6)
            name = "Unknown"
            authorized = False
            
            # Check if there's a match
            if True in matches:
                first_match_index = matches.index(True)  # Get the first match index
                
                # Try to get the name and authorization status
                name = known_names[first_match_index]
                authorized = bool(known_authorized_status[first_match_index])
            else:
                name = "Unknown"
                authorized = False
            
            # Scale face locations back up to the original frame size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4
            
            # Draw a rectangle around the face
            color = (0, 255, 0) if authorized else (0, 0, 255)
            cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
            
            # Draw label with name and status
            label = f"{name} - {'Authorized' if authorized else 'Unauthorized'}"
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, label, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
            
            # Log and save data to the database
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            save_detection_to_db(name, authorized, timestamp)

        except IndexError:
            # Skip this face without printing the error
            continue
        except Exception:
            # Skip any other errors silently without printing
            continue

    # Display the resulting frame
    cv2.imshow("Face Recognition", frame)

    # Wait for user input to quit
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release the webcam and close the OpenCV window
video_capture.release()
cv2.destroyAllWindows()
print("Face recognition stopped.")

Face detection started. Press 'q' to quit.
Data saved: Unknown, Unauthorized, 2024-11-20 10:13:18
Data saved: Unknown, Unauthorized, 2024-11-20 10:13:19
